In [5]:
import requests
import pandas as pd

# Function to read API key from a file
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

# Read the API key from the file
API_KEY = read_api_key('../API_key.txt')
location = '11.547301, 104.897463'  # Example: Latitude and Longitude of Phnom Penh, Cambodia
radius = 10000  # Radius in meters

# Define the types of educational buildings you want to search for
types = [
    'school', 'university', 'library', 'book_store', 
    'museum', 'training', 'academy', 'college', 
    'kindergarten', 'primary_school', 'secondary_school'
]

all_results = []

def fetch_places(place_type, location, radius, api_key, next_page_token=None):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={place_type}&key={api_key}"
    if next_page_token:
        url += f"&pagetoken={next_page_token}"
    
    response = requests.get(url)
    return response.json()

In [2]:
for place_type in types:
    data = fetch_places(place_type, location, radius, API_KEY)
    all_results.extend(data['results'])
    
    # Handle pagination if there are more results
    while 'next_page_token' in data:
        next_page_token = data['next_page_token']
        data = fetch_places(place_type, location, radius, API_KEY, next_page_token)
        all_results.extend(data['results'])

# Remove duplicates based on place_id
unique_results = {place['place_id']: place for place in all_results}.values()

# Prepare data for pandas DataFrame
data_for_df = []
for place in unique_results:
    data_for_df.append({
        'Name': place['name'],
        'Address': place.get('vicinity'),
        'Latitude': place['geometry']['location']['lat'],
        'Longitude': place['geometry']['location']['lng'],
        'Type': ', '.join(place['types'])
    })

In [3]:
len(data_for_df)

183

In [4]:
# Create pandas DataFrame
df = pd.DataFrame(data_for_df)
print(df)

# Optionally, save the DataFrame to a excel file
df.to_excel('educational_buildings.xlsx', index=False)

                                                  Name  \
0             CJCC (Cambodia-Japan Cooperation Center)   
1                               Angkor Computer Center   
2            SAS Santhormuk - Stanford American School   
3                 Sovannaphumi School, Tep Phan Campus   
4    Aii Language Center (Aii), Mao Tse Tong (QLH B...   
..                                                 ...   
178                        Chea Sim Samaki High School   
179                                          សាលាជប៉ុន   
180                                             School   
181                         Chamreun Phorl High School   
182  American Intercon School (AIS): MAO TSE TONG C...   

                                              Address   Latitude   Longitude  \
0    Rupp-CJCC, មហាវិថី សហពន្ធ័រុស្ស៊ី (១១០), ភ្នំពេញ  11.568929  104.893694   
1                        #95E0, Saint 164, Phnom Penh  11.563684  104.912819   
2               #197, St.146, Teuk laok 2, Phnom Penh  11.56498